In [1]:
import  numpy as np

In [2]:
# 定义基本域 GF(2)
GF_2 = GF(2) 
# 定义扩域 GF(2^8)
GF_2p8.<a> = GF(2^8, modulus=x^8+x^4+x^3+x+1)
# 定义扩域 GF(2^128)
GF_2p128.<b> = GF(2^128, modulus=x^128+x^17+x^9+x^7+1)


def to_2p128(x):
    return GF_2p128.from_integer(x.to_integer())
# 选定一组基(1,x^8,...,x^120),将F_{2^8}^{16}提升到F_{2^128}
def lift_to_GF2p128(x):
    res = GF_2p128(0)
    for i in range(16):
        temp1 = b^(i*8)
        temp2 = to_2p128(x[i])
        res += temp1 * temp2
    return res



In [ ]:
# test
test = np.array([GF_2p8.random_element() for _ in range(16)])
print(test)
print(lift_to_GF2p128(test))

In [3]:
# 将列向量x 过RE函数,x是字符串01字符串
def re(x):
    c = 8
    k = len(x) 
    x_block = [int(''.join(map(str, x[i:i+8][::-1])), 2) for i in range(0, len(x), 8)]
    y = [[1 if j == x_block[i]  else 0 for j in range(2**c)] for i in range(k//c)]
    return np.array(y)

# B n*(k/c·2^c) 
# n = 1280 
# k = 1280
# c = 8
# 1280*40960 
def gen_B():
    B = np.random.randint(0, 2000, size=(1280, 40960))
    B = (B == 1).astype(int)
    return B

def gen_x():
    x = np.random.randint(0, 2, size = 1280)
    return x

def AFS(B,x):
    y = re(x)
    temp = y.reshape(-1)
    return (B @ temp) % 2

B_matrix = gen_B()
# X_vec = gen_x()


In [ ]:
# test
test_B = gen_B()
print(test_B)
test_x = gen_x()
print("x:",test_x)
test_y = re(test_x)
test_res = AFS(test_B,test_x)
print(test_res)

In [6]:
# 建议不要使用此块代码,请运行下一个代码块,针对多项式优化后的版本


#将正则编码转换成布尔函数
# 一次性定义1280个符号变量
R_1280 = PolynomialRing(GF_2,'x',1280)
vars = R_1280.gens() # x0, x1, ..., x1279

def REtoBooleanFunc():
    binary_list = np.array([[int(bit) for bit in format(i, '08b')] for i in range(256)])
    res = np.zeros(40960,dtype=object)  # 预分配数组，160是1280/8
    for i in range(0, 1280, 8):
        tmp = np.ones(256,dtype=object)
        tmp = ((1 + binary_list[:, 7] + vars[i + 0])*(1 + binary_list[:, 6] + vars[i + 1])*(1 + binary_list[:, 5] + vars[i + 2])
                *(1 + binary_list[:, 4] + vars[i + 3])*(1 + binary_list[:, 3] + vars[i + 4])*(1 + binary_list[:, 2] + vars[i + 5])
                *(1 + binary_list[:, 1] + vars[i + 6])*(1 + binary_list[:, 0] + vars[i + 7]))
        res[i//8*256:(i//8+1)*256] = tmp# 存储结果
    return res

Boolean_vec = REtoBooleanFunc().reshape(-1,1)




In [ ]:
# 对于多形式进行优化

# 正则编码转换成布尔函数
R_opt = PolynomialRing(GF_2,'y',9)
vars_opt = R_opt.gens()

def REtoBool_opt():
    binary_list = np.array([[int(bit) for bit in format(i, '08b')] for i in range(256)])
    res = np.zeros(256,dtype=object)  # 预分配数组，160是1280/8
    res = ((1 + binary_list[:, 7] + vars_opt[0])*(1 + binary_list[:, 6] + vars_opt[1])*(1 + binary_list[:, 5] + vars_opt[2])
            *(1 + binary_list[:, 4] + vars_opt[3])*(1 + binary_list[:, 3] + vars_opt[4])*(1 + binary_list[:, 2] + vars_opt[5])
            *(1 + binary_list[:, 1] + vars_opt[6])*(1 + binary_list[:, 0] + vars_opt[7]))
    
    return res.reshape(256)

Bool_opt = REtoBool_opt()
print(Bool_opt)

In [ ]:
# test
test_x = gen_x()
test_y = re(test_x)
test_funcs = np.vectorize(lambda f: f(*test_x))
test_boolean = test_funcs(Boolean_vec)
test_y =test_y.reshape(-1,1)
print("false:",np.sum(test_y != test_boolean))

In [ ]:
# 建议不要使用此块代码,请运行下一个代码块,针对多项式优化后的版本
# AFS 的 布尔函数
FUNC = np.zeros(1280,dtype=object)
temp_TF = (B_matrix==1)
for i in range(1280):
    print(i)
    FUNC[i] = R_1280(0)
    for j in range(40960):
        if temp_TF[i][j]:
            FUNC[i] += Boolean_vec[j][0]
print(FUNC)


In [ ]:
# 不必运行

# 对于AFS的布尔函数进行优化

FUNC_opt = np.zeros((1280,160),dtype=object)
temp_TF = (B_matrix==1)
for i in range(1280):
    print(i)
    for j in range(160):
        FUNC_opt[i][j] = R_opt(0)
        for k in range(256):
            if temp_TF[i][j*256+k]:
                FUNC_opt[i][j] += Bool_opt[k]
print(FUNC_opt)

In [ ]:
# test
test_x = gen_x()
test_AFS = AFS(B_matrix,test_x)
test_funcs = np.vectorize(lambda f: f(*test_x))
test_FUNC = test_funcs(FUNC)

print("false:",np.sum(test_AFS != test_FUNC))

In [ ]:
# 建议不要使用此块代码,请运行下一个代码块,针对多项式优化后的版本

# AFS 按 deg
FUNC_f_ih = np.zeros((1280,9),dtype=object)
for i in range(1280):
    temp = FUNC[i].homogeneous_components()
    print(i)
    for j in range(9):
        FUNC_f_ih[i][j] = temp.get(j,R_1280(0)).change_ring(GF_2p128)

print(FUNC_f_ih)

FUNC_homo = np.zeros(1280,dtype=object)
for i in range(1280):
    print(i)
    FUNC_homo[i] = FUNC[i].homogenize().change_ring(GF_2p128)
print(FUNC_homo)

In [5]:
# 优化后的代码

def gen_func_homo(func_list):
    FUNC_opt_homo = np.zeros((1280,160),dtype=object)
    for i in range(1280):
        for j in range(160):
            if(func_list[i][j] == R_opt(1) or func_list[i][j] == R_opt(0)):
                FUNC_opt_homo[i][j] = func_list[i][j].change_ring(GF_2p128)
                continue
            temp_dict = func_list[i][j].homogeneous_components()
            if temp_dict.get(8,None) == None:
                for k in range(7,0,-1):
                    if temp_dict.get(k,None) != None:
                        func_list[i][j] = func_list[i][j] * vars_opt[8]^(8-k)
                        break
            FUNC_opt_homo[i][j] = func_list[i][j].homogenize(vars_opt[8]).change_ring(GF_2p128)
    return FUNC_opt_homo

In [ ]:
## 至此 setup部分结束
## 以下是dD_Rep部分

In [ ]:
# 此部分代码属于setup,为方便测试放在此处,这部分运行时间不应算入我们的方案中
FUNC_opt = np.zeros((1280,160),dtype=object)
temp_TF = (B_matrix==1)
for i in range(1280):
    print(i)
    for j in range(160):
        FUNC_opt[i][j] = R_opt(0)
        for k in range(256):
            if temp_TF[i][j*256+k]:
                FUNC_opt[i][j] += Bool_opt[k]

# 以下属于dD_Rep部分
def gen_witness():
    return gen_x()
def gen_func_dD_Rep(witness,B_mat):
    afs_res = AFS(B_mat,witness)
    for i in range(1280):
        FUNC_opt[i][0] += GF_2(afs_res[i])
    return 

In [ ]:
# test
test_witness = gen_witness()
gen_func_dD_Rep(test_witness,B_matrix)
for i in range(1280):
    temp = 0
    for j in range(160):
        temp += FUNC_opt[i][j](*(test_witness[j*8:(j+1)*8]),0) 
    if(temp != 0):
        print("err")
        break
else:
    print("pass")

In [ ]:
# 此部分代码属于setup,为方便测试放在此处,这部分运行时间不应算入我们的方案中
FUNC_opt = np.zeros((1280,160),dtype=object)
temp_TF = (B_matrix==1)
for i in range(1280):
    print(i)
    for j in range(160):
        FUNC_opt[i][j] = R_opt(0)
        for k in range(256):
            if temp_TF[i][j*256+k]:
                FUNC_opt[i][j] += Bool_opt[k]

# 以下代码属于dD-Rep
witness = gen_witness()
gen_func_dD_Rep(witness,B_matrix)
FUNC_opt_i_h = gen_func_homo(FUNC_opt)


In [ ]:
# test

print(FUNC_opt_i_h[0])
print(FUNC_opt_i_h[0][0](vars_opt[0],vars_opt[0],vars_opt[0],vars_opt[0],vars_opt[0],vars_opt[0],vars_opt[0],vars_opt[0],1))

In [8]:
# 需要运行,仅用于测试
l_vole = 2176 # 1280 + 7 * 128
G_C = np.array([1 for _ in range(16)]).reshape(1,16)
def vole():
    delta_vole = np.array([GF_2p8.random_element() for _ in range(16)])
    u_vole = np.zeros((l_vole,1),dtype=object)
    for i in range(l_vole):
        if randint(0,1) == 1:
            u_vole[i][0] = 1
        else:
            u_vole[i][0] = 0

    v_vole = np.zeros((l_vole,16),dtype=object)
    for i in range(l_vole):
        for j in range(16):
            v_vole[i][j] = GF_2p8.random_element()
    Q_vole = u_vole @ G_C @ np.diag(delta_vole) + v_vole

    return ((u_vole,v_vole),(Q_vole,delta_vole))
    

In [9]:
# dD-Rep
# λ = 128 Security level
# p = 2  Base field F_p
# q = 2^8 Extension field F_q
# r = 8  p =q^r
# τ = 16 Repetition for VOLEitH
l = 1280

deg = 8 

# 修改此处的t可以更改验证的规模.t代表一次性验证的多项式数量.在此测试中最大为1280.验证时间随t线性增长.建议使用较小的t验证.
t = 1280


# u \in F_2^{l+(d-1)rτ}
# v \in F_{2^r}^{(l+(d-1)rτ)*τ}
# w \in F_2^l
def round1(u,v,w):
    d = w - u[:l]
    u_lift = np.array([GF_2p128.from_integer(x) for x in u[l:]])
    v_lift = np.array([lift_to_GF2p128(x) for x in v])
    return (u_lift,v_lift,d)

def round2():
    challenge = np.array([GF_2p128.random_element() for _ in range(t)])
    return challenge


In [10]:
def round3(challenge,u_lift,v_lift,witness):
    A_i_j = np.zeros((t,deg),dtype = object)
    temp_list = v_lift[:l]+witness[:l]*vars_opt[8]
    for i in range(t):
        print(i)
        for j in range(deg):
            A_i_j[i][j] = GF_2p128(0)
        c_i_y = R_opt(0).change_ring(GF_2p128)
        for j in range(160):
            if (FUNC_opt_i_h[i][j] != R_opt(0) and FUNC_opt_i_h[i][j] != R_opt(1)):
                c_i_y += FUNC_opt_i_h[i][j](*(temp_list[j*8:(j+1)*8]),vars_opt[8])
            elif(FUNC_opt_i_h[i][j] == R_opt(1)):
                c_i_y += vars_opt[8]^8
            
        coeff = c_i_y.coefficients()
        temp_index_list = []
        h_coeff = c_i_y.homogeneous_components()
        for j in range(7,-1,-1):
            if h_coeff.get(j,None)!= None:
                temp_index_list.append(j)
        for j in range(len(temp_index_list)):
            A_i_j[i][temp_index_list[j]] = coeff[j]
    
    A_star_j = np.zeros(deg,dtype = object)
    G_i_x = PolynomialRing(GF_2p128,'zz',2)
    g_i_x = G_i_x(1)
    var_g_i_x = G_i_x.gens()
    for j in range(deg - 1):
        A_star_j[j] = GF_2p128(0)
        u_star = GF_2p128(0)
        v_star = GF_2p128(0)
        for i in range(128):
            u_star += u_lift[j*128+i] * b^i
            v_star += v_lift[l+j*128+i] * b^i
        g_i_x *= (v_star + u_star * var_g_i_x[0])
    
    coeff = g_i_x.coefficients()
    temp_index_list = []
    h_coeff = g_i_x.homogeneous_components()
    for j in range(7,-1,-1):
        if h_coeff.get(j,None)!= None:
            temp_index_list.append(j)
    for j in range(len(temp_index_list)):
        A_star_j[temp_index_list[j]] = coeff[j]

    a_verif = np.zeros(deg,dtype = object)
    for j in range(deg):
        a_verif[j] = A_star_j[j]
        for i in range(t):
            a_verif[j] += challenge[i] * A_i_j[i][j] 

    return a_verif

In [11]:
def ver(d,a_ver,delta,Q_vole,challenge):
    q_lift_star = np.zeros(l+deg-1,dtype=object)
    q_lift = np.zeros((deg-1)*128,dtype=object)
    Q_star = Q_vole[:l] + d.reshape(-1,1) @ np.array([1 for _ in range(16)]).reshape(1,16) @ np.diag(delta)
    delta_lift = lift_to_GF2p128(delta)
    for i in range(l):
        q_lift_star[i] = lift_to_GF2p128(Q_star[i])
    for i in range(l,l+(deg-1)*128):
        q_lift[i-l] = lift_to_GF2p128(Q_vole[i])
    
    for j in range(deg-1):
        q_lift_star[l+j] = GF_2p128(0)
        for i in range(128):
            q_lift_star[l+j] += q_lift[j*128+i] * b^i
    
    B_star = q_lift_star[l]
    for i in range(deg-2):
        B_star *= q_lift_star[l+i+1]
    

    c_i_delta = np.zeros(t,dtype=object)
    for i in range(t):
        c_i_delta[i] = GF_2p128(0)
        print(i)
        for j in range(160):
            if (FUNC_opt_i_h[i][j] != R_opt(0) and FUNC_opt_i_h[i][j] != R_opt(1)):
                c_i_delta[i] += FUNC_opt_i_h[i][j](*(q_lift_star[j*8:(j+1)*8]),delta_lift)
            elif(FUNC_opt_i_h[i][j] == R_opt(1)):
                c_i_delta[i] += delta_lift^8
    
    c_veri = B_star
    for i in range(t):
        c_veri += challenge[i] * c_i_delta[i] 
    
    c_ver = GF_2p128(0)
    for i in range(deg):
        c_ver += a_ver[i] * delta_lift^i
    
    return (c_ver == c_veri)

In [ ]:
# 至此,dD-Rep实现完成

# 以下是对协议的测试


##############################    test  ##################

In [12]:

# test 
prover,verifier = vole()
u_vole, v_vole = prover
Q_vole, delta_vole = verifier


In [13]:
u_lift,v_lift,d = round1(u_vole.reshape(2176),v_vole,witness)


In [ ]:
u_lift

In [ ]:
v_lift

In [ ]:
d

In [17]:
challenge = round2()

In [ ]:
challenge

In [ ]:
a_verifiction = round3(challenge,u_lift,v_lift,witness)

In [ ]:
a_verifiction

In [ ]:
res = ver(d,a_verifiction,delta_vole,Q_vole,challenge)

In [ ]:
res